In [1]:
import numpy as np
import itertools
import sympy as sp
import gurobipy as gp
from gurobipy import GRB

# Define the four functions

## Bin(INT) -> Bin(INT)
def identity(x):
    return x

def swap(x):
    return 1-x

def to0(_):
    return 0

def to1(_):
    return 1


basic_strategies = [identity, swap, to0, to1]
all_strategies = list(itertools.product(basic_strategies, repeat=2))

In [2]:
def generate_top(u):
    f_u, g_u = all_strategies[u]
    # print(f"f_{u} = {f_u.__name__} \ng_{u} = {g_u.__name__}\n")

    # value 1 at point(a,b) = (f_u(x),g_u(y)), zero everywhere else
    def middle_entry(x,y):
        # value 1 at (f_u(x),g_u(y)) zero everywhere else
        middle = np.zeros((2,2), dtype=int)
        middle[f_u(x), g_u(y)] = 1
        return middle

    top_level = np.array([
                [middle_entry(0,0), middle_entry(1,0)], 
                [middle_entry(0,1), middle_entry(1,1)]
                ]) 

    return top_level

# generate_top(15)

In [3]:
# declaring Gurobi variable W_i w/ i in {0,1,...,15}, and declaring v
m = gp.Model()
W = m.addVars(16, lb=0, ub=1, vtype=GRB.CONTINUOUS, name="W")
v = m.addVar(lb=0, ub=1, vtype=GRB.CONTINUOUS, name="v")
m.update()

Set parameter Username
Academic license - for non-commercial use only - expires 2025-01-09


In [4]:
def kron(a, b, x, y):
    return int(np.logical_xor(a, b) == np.logical_and(x, y))

# generates a Bell experiment distribution based on value of noise v0, 
# where high noise means classical system and low noise means quantum system.
def P_AB_giv_XY(a, b, x, y):
    return v*kron(a,b,x,y)/2 + (1-v)/4

In [5]:
internal_gurobi_expr = sum([W[i]*generate_top(i) for i in range(16)])


internal_gurobi_expr $[x,y,a,b]$  
P_AB_giv_XY $(a,b,x,y)$

Found summed strategies matrices for all U! (internal_gurobi_exp)

## Gurobi Constraints

In [6]:
# Add the constraint to the model
m.addConstr(W.sum() == 1, "sum(W_i) == 1")
for x,y, a,b in itertools.product(range(2), repeat=4):
    m.addConstr(internal_gurobi_expr[x,y,a,b] == P_AB_giv_XY(a,b,x,y), f"P({a},{b}|{x},{y}) == {internal_gurobi_expr[x,y,a,b]} == {P_AB_giv_XY(a,b,x,y)}")


# for cons in m.getConstrs():
#     print(cons)

# reset gurobi constraints
# m.remove(m.getConstrs())

m.update()

In [7]:
# maximum v?
m.setObjective(v, GRB.MAXIMIZE)
m.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11.0 (22621.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-11800H @ 2.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 17 rows, 17 columns and 96 nonzeros
Model fingerprint: 0x48c03462
Coefficient statistics:
  Matrix range     [3e-01, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 1e+00]
Presolve removed 8 rows and 1 columns
Presolve time: 0.01s
Presolved: 9 rows, 16 columns, 55 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0000000e+00   2.500000e+00   0.000000e+00      0s
       9    5.0000000e-01   0.000000e+00   0.000000e+00      0s

Solved in 9 iterations and 0.02 seconds (0.00 work units)
Optimal objective  5.000000000e-01


In [8]:
v

<gurobi.Var v (value 0.5)>